# Visual Search Paradigm

This task is based on classic visual search experiments in cognitive psychology. 

It tests how quickly and accurately you can detect a target (🔴 red circle in this case) among distractors. When the target "pops out" due to unique features, detection is fast (parallel search). When absent or similar to distractors, it takes longer (serial search). The task explores how attention works in simple vs. complex visual scenes.

You can try the demo yourself and see how your attention and reaction time compare across different trials.

In [1]:
import time, random
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

num_trials, grid_size = 10, 5
results, trial_index = [], 0
trials = ['present'] * 5 + ['absent'] * 5
random.shuffle(trials)
output = widgets.Output()
display(output)

def show_instructions():
    instructions = """
    <h2>🔍 Visual Search Task</h2>
    <p>Click <b>Start</b> to begin. In each trial, you’ll see a grid of colored shapes.
       Your task is to detect whether a <b>RED circle (🔴)</b> is present. Be fast and accurate!</p>
    """
    with output:
        clear_output()
        display(HTML(instructions))
        start_btn = widgets.Button(description="Start", button_style='success')
        start_btn.on_click(lambda _: run_trial())
        display(start_btn)

def generate_grid(condition):
    distractors = ['🟡', '🟢', '🔵', '🟤']
    total_cells = grid_size ** 2
    grid, target_added = [], False
    for _ in range(total_cells):
        if condition == 'present' and not target_added and random.random() < 0.1:
            grid.append('🔴')
            target_added = True
        else:
            grid.append(random.choice(distractors))
    if condition == 'present' and not target_added:
        grid[random.randint(0, total_cells - 1)] = '🔴'
    rows = [' '.join(grid[i:i+grid_size]) for i in range(0, total_cells, grid_size)]
    return '<br>'.join(rows)

def run_trial():
    global trial_index, start_time
    if trial_index >= num_trials:
        return show_results()
    
    condition = trials[trial_index]
    grid_html = generate_grid(condition)
    content = f"""
    <h4>Trial {trial_index + 1}</h4>
    <div style="font-size:28px; text-align:center; line-height:2.5; letter-spacing:15px;">
        {grid_html}
    </div>
    <p style="text-align:center;">Is the red circle (🔴) present?</p>
    """
    with output:
        clear_output()
        display(HTML(content))
        btn_yes = widgets.Button(description="Target Present", button_style='danger', layout=widgets.Layout(width='150px'))
        btn_no = widgets.Button(description="Target Absent", button_style='info', layout=widgets.Layout(width='150px'))
        btn_yes.on_click(lambda _: record_response('present', condition))
        btn_no.on_click(lambda _: record_response('absent', condition))
        display(widgets.HBox([btn_yes, btn_no]))
    start_time = time.time()

def record_response(user_resp, correct_cond):
    global trial_index
    rt = round(time.time() - start_time, 3)
    results.append({
        'Trial': trial_index + 1,
        'Condition': correct_cond.capitalize(),
        'Response': user_resp.capitalize(),
        'Correct': (user_resp == correct_cond),
        'RT (s)': rt
    })
    trial_index += 1
    run_trial()

def show_results():
    present_trials = [r for r in results if r['Condition'] == 'Present']
    absent_trials = [r for r in results if r['Condition'] == 'Absent']
    avg_rt = lambda data: round(sum(r['RT (s)'] for r in data)/len(data), 3) if data else 0
    correct_pct = lambda data: f"{round(100 * sum(r['Correct'] for r in data) / len(data), 1)}%" if data else "0%"
    results_html = f"""
    <h2>✅ Task Complete!</h2>
    <p><b>Target Present Trials:</b> {correct_pct(present_trials)} correct | Avg RT: {avg_rt(present_trials)} sec</p>
    <p><b>Target Absent Trials:</b> {correct_pct(absent_trials)} correct | Avg RT: {avg_rt(absent_trials)} sec</p>
    <hr>
    <p>🧠 Did you find it easier or harder when the target was absent?</p>
    """
    with output:
        clear_output()
        display(HTML(results_html))

# Launch the task
show_instructions()


Output()